In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.dates as mdates
import scipy.stats as sp
import seaborn as sns
from scipy.stats.stats import pearsonr
from datetime import datetime
from sklearn.model_selection import learning_curve
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import metrics
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from numpy import concatenate
from matplotlib import pyplot
from pandas import DataFrame
from tensorflow.keras.optimizers import Adam

In [2]:
df_er = pd.read_excel (r'/Users/brianyu/Desktop/Term Project/Data_COMP5212project_brian.xlsx', sheet_name='ER')

In [3]:
df_er['datetime']=pd.to_datetime(df_er['Date'], format='%Y%m%d', errors='raise')

In [4]:
df_er=df_er.drop(columns='Date')

In [5]:
df_var = pd.read_excel (r'/Users/brianyu/Desktop/Term Project/Data_COMP5212project_brian.xlsx', sheet_name='Variables')

In [6]:
df_var['datetime']=pd.to_datetime(df_var['Date'], format='%Y%m%d', errors='raise')

In [7]:
df_var=df_var.drop(columns='Date')

In [8]:
df_all=df_er.set_index('datetime',drop=False).join(df_var.set_index('datetime'))

In [9]:
df_usdeu=df_all[['EUR_USD', 'SP500', 'EuroStoxx50','GoldPrice', 'BrentOilFutures', 'CPI_USA', 'CPI_EU', 'STINT_USA', 'STINT_EA19', 'LTINT_USA', 'LTINT_EA19', 'DLTINT_USEA']]

In [10]:
df_usdeu.head()

,EUR_USD,SP500,EuroStoxx50,GoldPrice,BrentOilFutures,CPI_USA,CPI_EU,STINT_USA,STINT_EA19,LTINT_USA,LTINT_EA19,DLTINT_USEA
datetime,,,,,,,,,,,,
1999-01-01,1.1366,1279.64,NaN,286.3,11.35,69.31992,NaN,4.89,3.1321,4.72,3.8220,0.8980
1999-02-01,1.1025,1238.33,NaN,287.5,10.88,69.40430,NaN,4.90,3.0934,5.00,3.9742,1.0258
1999-03-01,1.0770,1286.37,NaN,279.8,15.24,69.61526,NaN,4.91,3.0467,5.23,4.1810,1.0490
1999-04-01,1.0570,1335.18,NaN,286.8,16.57,70.12155,NaN,4.88,2.6965,5.18,4.0407,1.1393
1999-05-01,1.0423,1301.84,NaN,270.4,15.20,70.12155,NaN,4.92,2.5790,5.54,4.2049,1.3351


In [11]:
def readTrain():
    train=pd.read_excel(r'/Users/brianyu/Desktop/Term Project/Data_usdeu.xlsx', sheet_name='Variables')
    return train

In [12]:
train = readTrain()

In [13]:
train.head()

,Date,SP500,EuroStoxx50,GoldPrice,BrentOilFutures,CPI_USA,CPI_EU,STINT_USA,STINT_EA19,LTINT_USA,LTINT_EA19,DLTINT_USEA,EUR_USD
0,1999-01-01,1279.64,NaN,286.3,11.35,69.31992,NaN,4.89,3.1321,4.72,3.8220,0.8980,1.1366
1,1999-02-01,1238.33,NaN,287.5,10.88,69.40430,NaN,4.90,3.0934,5.00,3.9742,1.0258,1.1025
2,1999-03-01,1286.37,NaN,279.8,15.24,69.61526,NaN,4.91,3.0467,5.23,4.1810,1.0490,1.0770
3,1999-04-01,1335.18,NaN,286.8,16.57,70.12155,NaN,4.88,2.6965,5.18,4.0407,1.1393,1.0570
4,1999-05-01,1301.84,NaN,270.4,15.20,70.12155,NaN,4.92,2.5790,5.54,4.2049,1.3351,1.0423


In [14]:
def Features(train):
    train["Date"] = pd.to_datetime(train["Date"])
    train["year"] = train["Date"].dt.year
    train["month"] = train["Date"].dt.month
    train["date"] = train["Date"].dt.day
    train["day"] = train["Date"].dt.dayofweek
    return train

In [15]:
train_fea = Features(train)

In [16]:
train_fea.head()

,Date,SP500,EuroStoxx50,GoldPrice,BrentOilFutures,CPI_USA,CPI_EU,STINT_USA,STINT_EA19,LTINT_USA,LTINT_EA19,DLTINT_USEA,EUR_USD,year,month,date,day
0,1999-01-01,1279.64,NaN,286.3,11.35,69.31992,NaN,4.89,3.1321,4.72,3.8220,0.8980,1.1366,1999,1,1,4
1,1999-02-01,1238.33,NaN,287.5,10.88,69.40430,NaN,4.90,3.0934,5.00,3.9742,1.0258,1.1025,1999,2,1,0
2,1999-03-01,1286.37,NaN,279.8,15.24,69.61526,NaN,4.91,3.0467,5.23,4.1810,1.0490,1.0770,1999,3,1,0
3,1999-04-01,1335.18,NaN,286.8,16.57,70.12155,NaN,4.88,2.6965,5.18,4.0407,1.1393,1.0570,1999,4,1,3
4,1999-05-01,1301.84,NaN,270.4,15.20,70.12155,NaN,4.92,2.5790,5.54,4.2049,1.3351,1.0423,1999,5,1,5


In [17]:
def normalize(train):
    train = train.drop(["Date", "date"], axis=1)
    train_norm = train.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
    return train_norm

In [18]:
train_norm = normalize(train_fea)

In [19]:
train_norm=train_norm.dropna()

In [20]:
train_norm.head()

,SP500,EuroStoxx50,GoldPrice,BrentOilFutures,CPI_USA,CPI_EU,STINT_USA,STINT_EA19,LTINT_USA,LTINT_EA19,DLTINT_USEA,EUR_USD,year,month,day
56,-0.214933,-0.300321,-0.333837,-0.267516,-0.309355,-0.343877,-0.146929,0.091018,0.144071,0.183710,-0.046852,-0.047496,-0.299385,0.230690,-0.501253
57,-0.197137,-0.229682,-0.334352,-0.266818,-0.311411,-0.339556,-0.143908,0.090364,0.147382,0.196871,-0.060460,-0.057886,-0.299385,0.321599,-0.167920
58,-0.194702,-0.207828,-0.327319,-0.261002,-0.316551,-0.337251,-0.142397,0.093085,0.149038,0.219271,-0.089444,-0.001837,-0.299385,0.412509,0.332080
59,-0.177238,-0.156500,-0.316511,-0.247663,-0.318606,-0.330049,-0.143908,0.090841,0.144071,0.205144,-0.076901,0.079366,-0.299385,0.503418,-0.501253
60,-0.170993,-0.125563,-0.324231,-0.255341,-0.309355,-0.331777,-0.149950,0.080807,0.124203,0.187993,-0.081913,0.063098,-0.253930,-0.496582,-0.001253


In [21]:
def buildTrain(train, pastDay=30, futureDay=5):
    X_train, Y_train = [], []
    for i in range(train.shape[0]-futureDay-pastDay):
        X_train.append(np.array(train.iloc[i:i+pastDay]))
        Y_train.append(np.array(train.iloc[i+pastDay:i+pastDay+futureDay]["EUR_USD"]))
        return np.array(X_train), np.array(Y_train)

In [22]:
def shuffle(X,Y):
    np.random.seed()
    randomList = np.arange(X.shape[0])
    np.random.shuffle(randomList)
    return X[randomList], Y[randomList]

In [23]:
def splitData(X,Y,rate):
    X_train = X[int(X.shape[0]*rate):]
    Y_train = Y[int(Y.shape[0]*rate):]
    X_val = X[:int(X.shape[0]*rate)]
    Y_val = Y[:int(Y.shape[0]*rate)]
    return X_train, Y_train, X_val, Y_val

In [24]:
train = readTrain()
train_fea = Features(train)
train_norm = normalize(train_fea)
X_train, Y_train = buildTrain(train_norm, 30, 5)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

In [25]:
print('X_train shape:',X_train.shape)
print('y_train shape:',Y_train.shape)
print('X_test shape:',X_val.shape)  
print('y_test shape:',Y_val.shape)
#why the shape is 1? it should be total number of data? i.e. 209?

X_train shape: (1, 30, 15)
y_train shape: (1, 5)
X_test shape: (0, 30, 15)
y_test shape: (0, 5)


In [26]:
def buildOneToOneModel(shape):
    model = Sequential()
    model.add(LSTM(10, input_length=shape[1], input_dim=shape[2],return_sequences=True))
    # output shape: (1, 1)
    model.add(TimeDistributed(Dense(1)))    # or use model.add(Dense(1))
    model.compile(loss="mse", optimizer="adam")
    model.summary()
    return model

In [27]:
train = readTrain()
train_fea = Features(train)
train_norm = normalize(train_fea)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 1, 1)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,np.newaxis]
Y_val = Y_val[:,np.newaxis]

model = buildOneToOneModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 10)             1040      
_________________________________________________________________
time_distributed (TimeDistri (None, 1, 1)              11        
Total params: 1,051
Trainable params: 1,051
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: nan
Epoch 2/1000
1/1 [==============================] - 0s 30ms/step - loss: nan
Epoch 3/1000
1/1 [==============================] - 0s 29ms/step - loss: nan
Epoch 4/1000
1/1 [==============================] - 0s 30ms/step - loss: nan
Epoch 5/1000
1/1 [==============================] - 0s 29ms/step - loss: nan
Epoch 6/1000
1/1 [==============================] - 0s 31ms/step - loss: nan
Epoch 7/1000
1/1 [==============================] - 0s 3